# What deos the code do?
- It extracts PEFs with single attendant and desired statuses from "*** Report" restricted for one business area.
- Then logs in to *** with given credentials.
- Opens "Talepler" page.
- Puts the first a few characters of the first PEF No. on the PEF list to call PEFs of the business area.
- Looks whether PEF numbers on RES exist on the PEF list created at the beginning.
- If it exists enters to the PEF page and extracts and stores the notes there.

In [1]:
#Import packages for data cleaning
import pandas as pd
import os

In [3]:
# Determine file name and folder path and import the data
folder_path = r"***"
file_name = "***"
file_path = os.path.join(folder_path, file_name)

raw_df = pd.read_excel(file_path, header=0)

In [5]:
# Get PEFs with single attendants to loop over them on RES
single_df = raw_df[['request_no', 'bidder_company', 'status']].groupby(
    "request_no").agg(attendant_no = ("bidder_company", "nunique"),
                      status = ("status", "first")).reset_index()

# Determine the correct statuses to keep them only
statuses = ['Kısmi Sipariş Geçildi', 'Ön Anlaşma Bekleniyor', 'Sipariş Geçildi', 
            'Sipariş Geçilmesi Bekleniyor', 'Sipariş Tamamlandı', "Sipariş TDFde Tamamlandı"]

single_df = single_df[single_df["attendant_no"]==1 & single_df["status"].isin(statuses)]
single_df = single_df["request_no"]

In [7]:
# Import packages for scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
import time
import pyautogui # It is gonna be used to prevent windows get locked

In [9]:
# Type the user name and the password for RES
user_name = "***"
password = "***"

# Get the first a few characters of the PEF numbers to call all PEFs of the construction site 
tdf_no = list(single_df)[0][:9]

In [92]:
# Opens to use Chrome to execute the code below
options = Options()
options.add_argument("--force-device-scale-factor=0.8")  # 70% zoom
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# Opens the given website
driver.get("***")

In [ ]:
try:
    # Put mail & password, then login
    email_field = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.NAME, "email")))
    email_field.send_keys(user_name)
    
    WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.NAME, "password")))
    pw_field = driver.find_element(By.NAME, "password")
    pw_field.send_keys(password)
    
    login_button = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.ID, "login_signin_submit")))
    login_button.click()
    
    # Click "Talep", then "Talepler"
    talep_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, 
                                                "/html/body/div[2]/div/div[2]/div[1]/div[2]/div/ul/li[2]/a")))
    talep_button.click()
    
    talepler_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, 
                                                "/html/body/div[2]/div/div[2]/div[1]/div[2]/div/ul/li[2]/div/ul/li[3]/a")))
    talepler_button.click()
    
    # Put the tdf_no & hit Enter
    tdf_no_field = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, 
                                                "/html/body/div[2]/div/div[2]/div[2]/div/div[3]/div[2]/div/div/div/div/div/div/div[1]/div[4]/table/thead/tr[2]/th[1]/input")))
    tdf_no_field.click()                                                
    tdf_no_field.send_keys(tdf_no)
    tdf_no_field.send_keys(Keys.ENTER)
    time.sleep(10)

    #Find how many pages are there    
    elements = driver.find_elements(By.XPATH, 
                                    "/html/body/div[2]/div/div[2]/div[2]/div/div[3]/div[2]/div/div/div/div/div/div/div[2]/div[2]/div[2]/ul/*")  
    last_page = int(elements[-2].text)
    print("The last page is:", last_page)
                                            
    # Create loop for all pages of the table
    all_data = []
    for page in range(1, last_page + 1):
        print(f"\n--- Loading page {page} ---")
            
        # Wait for table body to load
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH, 
                       "//table[@id='procurement-requests__table']/tbody/*")))
        table_body = driver.find_element(By.XPATH, "//table[@id='procurement-requests__table']/tbody")
    
        # Get all <tr> elements in the current page
        rows = table_body.find_elements(By.TAG_NAME, "tr")
        print(f"Found {len(rows)} rows on page {page}")

        # Loop over each row
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            talep_no = cells[0].text.strip()
            
            if talep_no in list(single_df):
                print(talep_no, " is selected and its notes will be extracted.")
                try:
                    # Click "Geçmiş" button (last cell in the row)
                    geçmiş_button = cells[-2].find_element(By.TAG_NAME, "a")
                    driver.execute_script("arguments[0].scrollIntoView(true);", geçmiş_button)
                    geçmiş_button.click()
            
                    # Wait for popup to appear and click inner button
                    popup_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "/html/body/div[24]/div/div/div[2]/div/div/div/div/div[3]/div/a")))
                    popup_button.click()
            
                    # Handle new window
                    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                    main_window = driver.current_window_handle
                    all_windows = driver.window_handles
                    new_window = [w for w in all_windows if w != main_window][0]
                    driver.switch_to.window(new_window)
            
                    # Wait for the notes in the new page
                    WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "kt-notes__body")))
                    notes_table = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[3]/div[2]/div")
                
                    # Scrape each notes, personnel and time. Then store them in all_data
                    notes_table_data = notes_table.find_elements(By.CLASS_NAME, 'kt-notes__content')

                    all_pef_notes = ""
                    for data in notes_table_data:
                        note_taker = data.find_element(By.CLASS_NAME, "kt-notes__title").text
                        note_time = data.find_element(By.CLASS_NAME, "kt-badge").text
                        the_note = data.find_element(By.CLASS_NAME, "kt-notes__body").text
                        pef_note = "_".join([note_taker, note_time, the_note])
                        all_pef_notes = "***".join([all_pef_notes, pef_note]) if all_pef_notes else pef_note
             
                    # Close the new window, return to main window and close the pop-up
                    driver.close()
                    driver.switch_to.window(main_window)
                    driver.switch_to.active_element.send_keys(Keys.ESCAPE)                

                except Exception as e:
                    print(f"Error while processing talep_no {talep_no}: {e}")
                    all_pef_notes = "THE DATA COULD NOT BE EXTRACTED!" if not all_pef_notes else all_pef_notes + "___NEEDS TO BE CHECKED!"
                    # Close new window only if it exists
                    if len(driver.window_handles) > 1:
                        for handle in driver.window_handles:
                            if handle != main_window:
                                driver.switch_to.window(handle)
                                driver.close()
                        driver.switch_to.window(main_window)
                        driver.switch_to.active_element.send_keys(Keys.ESCAPE)                
                    continue

                finally:
                    # Store the data
                    all_data.append({"pef_no": talep_no, "note": all_pef_notes})

        # Click to sonraki_button to go tho the next page
        sonraki_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//ul[@class='pagination']/li[last()]/a")))
        driver.execute_script("arguments[0].scrollIntoView(true);", sonraki_button)
        sonraki_button.click()
        time.sleep(10)
        pyautogui.press('shift') # It prevents windows to get locked
    
finally:
    input("Press Enter to close the browser...")
    driver.close()


In [97]:
# Write the output to an excel
pd.DataFrame(all_data).to_excel("trial1.xlsx", index=False)